In [1]:
import keras

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np
import glob

In [3]:
from utils.schemas import schema_train_4


# Load Data

Para cargar los datos con pandas, voy a usar la misma forma que usamos en LightGBM

In [4]:
!ls ../../../data/

data_4.zip  df_num    test_final_4  train_final_4
df_cat	    test.csv  train.csv     train_model_little.csv


In [5]:
RUTA_TRAIN = "../../../data/train_model_little.csv"
RUTA_TEST = ""

In [ ]:
# print('Cargando datos del TRAIN')
# path = RUTA_TRAIN
# allFiles = glob.glob(path + "/*.csv")
# list_ = []
# for file_ in allFiles:
#     df = pd.read_csv(file_)
#     df = (df.fillna(-1)).astype(schema_train_4)
#     list_.append(df)

# train = pd.concat(list_, axis = 0, ignore_index= True)

# train.head()

In [ ]:
df = pd.read_csv(RUTA_TRAIN)
df = (df.fillna(0)).astype(schema_train_4)

train = df

train.head()

In [6]:
df = pd.read_csv(RUTA_TRAIN)
# df = (df.fillna(0)).astype(schema_train_4)
df = (df).astype(schema_train_4)


train = df

# train.head()

In [7]:
len(df[df[categoricas[0]] == -1].values)

NameError: name 'categoricas' is not defined

In [18]:
columns_bad = []
for i in categoricas:
    if(len(df[df[i] == -1].values)) > 0:
        print(i)
#         columns_bad.append(i)
    
        

CityIdentifier
OrganizationIdentifier
IeVerIdentifier
RtpStateBitfield


In [19]:
for i in columns_bad:
     df.loc[df[i] == -1,i] = 99
    

In [ ]:
df.loc[df['CityIdentifier'] == -1,'CityIdentifier'] = 9999999

In [ ]:
df.loc[df['OrganizationIdentifier'] == -1]

In [ ]:
df['RtpStateBitfield']

In [ ]:
df.columns

## Generate X and Y 

In [7]:
drop_version = ['AvSigVersion_index', 'EngineVersion_index', 'Census_OSVersion_index', 'AppVersion_index']

sel_cols = [c for c in train.columns if c not in ['MachineIdentifier',
                                                      'HasDetections',
                                                      'Census_DeviceFamily_Windows.Server',
                                                      'Census_DeviceFamily_Windows.Desktop'
                                                     ]+drop_version]

In [8]:
X = train.loc[:, sel_cols]
y = train.loc[:,'HasDetections']

## Train y test

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [11]:
print("X_train :"+str(X_train.shape))

print("X_test :"+str(X_test.shape))


print("y_train :"+str(y_train.shape))

print("y_test :"+str(y_test.shape))

X_train :(89188, 117)
X_test :(22298, 117)
y_train :(89188,)
y_test :(22298,)


# Keras Model

In [12]:


from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Embedding,Input,Concatenate,Reshape

# from keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import cross_val_score
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline

In [ ]:
dimension = X_train.shape[1]

In [ ]:
dimension

## Metrics

### AUC 1

Easy

In [ ]:
from sklearn.metrics import roc_auc_score
import tensorflow as tf

def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

### AUC 2

For Keras.

A little more complex

In [ ]:
from keras import callbacks
from sklearn.metrics import roc_auc_score

class printAUC(callbacks.Callback):
    def __init__(self, X_train, y_train):
        super(printAUC, self).__init__()
        self.bestAUC = 0
        self.X_train = X_train
        self.y_train = y_train
        
    def on_epoch_end(self, epoch, logs={}):
        pred = self.model.predict(np.array(self.X_train))
        auc = roc_auc_score(self.y_train, pred)
        print("Train AUC: " + str(auc))
        pred = self.model.predict(self.validation_data[0])
        auc = roc_auc_score(self.validation_data[1], pred)
        print ("Validation AUC: " + str(auc))
        if (self.bestAUC < auc) :
            self.bestAUC = auc
            self.model.save("bestNet.h5", overwrite=True)
        return

## EarlyStopping

In [ ]:
earlyStopping = keras.callbacks.EarlyStopping(monitor='auroc', patience=20, verbose=2,min_delta=0.01, mode='auto')

# Model work



## Embedding Layers

Primero, identificamos las variables categoricas

In [13]:
categoricas = ['ProductName_index','EngineVersion_0_index','EngineVersion_1_index','Census_OSVersion_0_index','Census_OSVersion_1_index','AppVersion_0_index','AppVersion_1_index','AvSigVersion_0_index','AvSigVersion_1_index','OsVer_0_index','OsVer_1_index','Platform_index','Processor_index','OsPlatformSubRelease_index','OsBuildLab_index','SkuEdition_index','PuaMode_index','SmartScreen_index','Census_MDC2FormFactor_index','Census_DeviceFamily_index','Census_ProcessorClass_index','Census_PrimaryDiskTypeName_index','Census_ChassisTypeName_index','Census_PowerPlatformRoleName_index','Census_InternalBatteryType_index','Census_OSArchitecture_index','Census_OSBranch_index','Census_OSEdition_index','Census_OSSkuName_index','Census_OSInstallTypeName_index','Census_OSWUAutoUpdateOptionsName_index','Census_GenuineStateName_index','Census_ActivationChannel_index','Census_FlightRing_index','OsVer_index','DefaultBrowsersIdentifier','AVProductStatesIdentifier','CountryIdentifier','CityIdentifier','OrganizationIdentifier','GeoNameIdentifier','LocaleEnglishNameIdentifier','IeVerIdentifier','Census_OEMNameIdentifier','Census_OEMModelIdentifier','Census_ProcessorManufacturerIdentifier','Census_ProcessorModelIdentifier','Census_OSInstallLanguageIdentifier','Census_OSUILocaleIdentifier','Census_FirmwareManufacturerIdentifier','Census_FirmwareVersionIdentifier','Wdft_RegionIdentifier','RtpStateBitfield','prediction_2','prediction_4','prediction_8','prediction_16','prediction_32','prediction_64']

booleanas = ['Census_IsSecureBootEnabled','Census_IsWIMBootEnabled','Census_IsVirtualDevice','Census_IsTouchEnabled','Census_IsPenCapable','Census_IsAlwaysOnAlwaysConnectedCapable','Wdft_IsGamer','Census_IsPortableOperatingSystem','Census_IsFlightingInternal','Census_IsFlightsDisabled','IsProtected','IsBeta','IsSxsPassiveMode','AVProductsInstalled','AVProductsEnabled','HasTpm','Census_HasOpticalDiskDrive','Census_ThresholdOptIn','Census_InternalBatteryType_informed','Firewall','SMode','AutoSampleOptIn']

numericas = ['OsBuild','OsSuite','UacLuaenable','Census_ProcessorCoreCount','Census_PrimaryDiskTotalCapacity','Census_SystemVolumeTotalCapacity','Census_TotalPhysicalRAM','Census_InternalPrimaryDiagonalDisplaySizeInInches','Census_InternalPrimaryDisplayResolutionHorizontal','Census_InternalPrimaryDisplayResolutionVertical','Census_InternalBatteryNumberOfCharges','Census_OSBuildNumber','Census_OSBuildRevision','OsBuildLab_diff','std_diff_DateOsBuildLab','AvSigVersion_diff','std_diff_AvSigVersion','OSVersion_diff','std_diff_OSVersion','max_OsBuildLab_diff','max_AvSigVersion_diff','max_OSVersion_diff','ratio_OsBuildLab_diff','ratio_AvSigVersion_diff','ratio_OSVersion_diff','count(DISTINCT AvSigVersion_Name)','count(DISTINCT AvSigVersion_Type)','count(DISTINCT AvSigVersion_AlertLevel)','count1','count2','count3','count4','count5','count6','count7','count8']

In [14]:
for i in list(X_train.columns):
    if(len(X_train[X_train[i] == -1].values)) > 0:
        print(i)
        X_train.loc[X_train[i] == -1,i] = 99
#         columns_bad.append(i)

RtpStateBitfield
CityIdentifier
OrganizationIdentifier
IeVerIdentifier


/home/rde/stuff_workspace/2_Kaggle/kaggle_microsoft/enviroment/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


UacLuaenable
Census_InternalBatteryNumberOfCharges
count(DISTINCT AvSigVersion_Name)
count(DISTINCT AvSigVersion_Type)
count(DISTINCT AvSigVersion_AlertLevel)


In [15]:
for i in list(X_train.columns):
    if(len(X_train[X_train[i] == -1].values)) > 0:
        print(i)


```py
input_numeric = Input(shape=(24,))
embedding_numeric = Dense(16)(input_numeric) 
inputs.append(input_numeric)
embeddings.append(embedding_numeric)

x = Concatenate()(embeddings)
x = Dense(80, activation='relu')(x)
x = Dropout(.35)(x)
x = Dense(20, activation='relu')(x)
x = Dropout(.15)(x)
x = Dense(10, activation='relu')(x)
x = Dropout(.15)(x)
output = Dense(1, activation='sigmoid')(x)
```

In [47]:
realcat=[]
ins = []
outs = []
for i in categoricas:
    if X_train[i].nunique() < 300:
        realcat.append(i)
        x = Input(shape=(1,),name='input_'+i)
        ins.append(x)
        x = Embedding(X_train[i].max()+1, X_train[i].max()+1, input_length=1,name='embedding_'+i)(x)
        x = Reshape(name='reshape_'+i,target_shape=(X_train[i].max()+1,))(x)
        outs.append(x)
    
# for i in booleanas:
#     x = Input(shape=(1,))
#     ins.append(x)   
#     x = Embedding(df[i].nunique(), df[i].nunique(), input_length=1)(x)
#     x = Reshape(target_shape=(df[i].nunique(),))(x)
#     outs.append(x)

## Network Architecture

In [49]:
primer_input_len = len(numericas+booleanas)
primer_input_len

58

In [50]:
len_batch_size = int(X_train.shape[0]/1000)
epochs_to_train = 15

In [51]:
x = Input(shape=(primer_input_len, ),name="Categoricas_Booleanas")
ins.append(x)
# x = Dense(50)(x) 
# outs.append(x)

x = Concatenate()(outs)
#then continue making NN
x = Dense(100, activation='relu')(x)
x = Dense(100, activation='relu')(x)
# x = Dense(1, activation='sigmoid')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=ins, outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam',metrics = ['acc'])
#model.fit( [X_train[col] for col in ListCategorical] + [X_train[ListNonCategorical]], 

### Save model to view

In [30]:
# import keras.utils.vis_utils

In [45]:
# model.save("modelo2.h5")

## FIT!

In [40]:
model.fit( [X_train[col] for col in realcat] +  [X_train[numericas+booleanas]], 
           y_train,
          epochs=epochs_to_train)
#           validation_data = ([X_test[col] for col in realcat] +  [X_test[numericas+booleanas]],y_test))

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
 4672/89188 [>.............................] - ETA: 5:55 - loss: 0.6940 - acc: 0.5116

KeyboardInterrupt: 

In [52]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ProductName_index (InputL (None, 1)            0                                            
__________________________________________________________________________________________________
input_EngineVersion_0_index (In (None, 1)            0                                            
__________________________________________________________________________________________________
input_EngineVersion_1_index (In (None, 1)            0                                            
__________________________________________________________________________________________________
input_Census_OSVersion_0_index  (None, 1)            0                                            
__________________________________________________________________________________________________
input_Cens

In [44]:
from ann_visualizer.visualize import ann_viz

ann_viz(model, title="NN sin densidad")

ValueError: ANN Visualizer: Layer not supported for visualizing

# Ejemplo de Modelos

Este es un "esquema" que me dijo un chaval __TODO: PONER EL NOMBRE__ para hacer los embeddings.

```py
x = Input(shape=(len(ListNonCategorical), ))
ins.append(x)
outs.append(x)

x = Concatenate(outs)
#then continue making NN
x = Dense(100)(x)
x = Dense(100)(x)
x = Dense(1)(x)
```

# Old Model

Este modelo ya no se usa.

In [ ]:
model = Sequential()

model.add(Dense(256, input_dim=dimension, kernel_initializer='uniform', activation='relu'))
model.add(Dense(512))
model.add(Dense(128))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam',metrics = ['acc'])

model.summary()

## Fit Model

In [ ]:
model.fit(X_train, y_train,
          epochs=epochs_to_train,
          batch_size=len_batch_size,
            validation_data = (X_test,y_test))
#              callbacks=[earlyStopping])

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=int(X_train.shape[0]/100),verbose=1)

In [ ]:
score

# Graphs

In [ ]:
model.history.history.keys()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(model.history.history['loss'],label="Loss")
plt.plot(model.history.history['auroc'],label="acc")
plt.legend()

In [ ]:
plt.plot(model.history.history['acc'])

In [ ]:
plt.plot(model.history.history['auroc'])

# Utils

## Clean Memory

In [ ]:
del model

In [ ]:
import gc
gc.collect()

# Prediction

In [ ]:
print('Cargando datos del TEST')
path = '../../data/test_final_3'
allFiles = glob.glob(path + "/*.csv")
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_)
    df = (df.fillna(-1)).astype(schema_train_3)
    list_.append(df)


test = pd.concat(list_, axis = 0, ignore_index = True).fillna(-1)

sel_cols = [c for c in test.columns if c not in ['MachineIdentifier',
                                                 'HasDetections',
                                                 'Census_DeviceFamily_Windows.Server',
                                                 'Census_DeviceFamily_Windows.Desktop'
                                                 ]
            ]

X_test = test.loc[:, sel_cols]

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions

In [ ]:
predf= pd.DataFrame(predictions,columns=['HasDetections'])

In [ ]:
pd.concat([X_machines,predf],axis=1).to_csv('keras_model.csv', index=None)

In [ ]:
X_machines = test.loc[:, 'MachineIdentifier']


df_prds = pd.DataFrame({'MachineIdentifier': X_machines, 'HasDetections': predictions})

df_prds.to_csv('keras_model.csv', index=None)

In [ ]:
del predictions

### Save train in one file

In [ ]:
train.to_csv("train_uniq.csv")

In [ ]:
test.to_csv("test_uniq.csv")